In [1]:
import random
import glob
import json
from tqdm import tqdm

import torch
from torch.utils.data import DataLoader
from transformers import BertJapaneseTokenizer, BertModel
import pytorch_lightning as pl


import numpy as np
import pandas as pd
import matplotlib.pylab as plt
import math
import csv
import tqdm

import os
import sys
import re
import pickle
import json

In [2]:
#MODEL_NAME="izumi-lab/bert-small-japanese-fin"
MODEL_NAME = 'cl-tohoku/bert-base-japanese-whole-word-masking'

In [3]:
# 7-12
# トークナイザのロード
tokenizer = BertJapaneseTokenizer.from_pretrained(MODEL_NAME)

In [4]:
# 7-4
class BertForSequenceClassificationMultiLabel(torch.nn.Module):
    
    def __init__(self, model_name, num_labels):
        super().__init__()
        # BertModelのロード
        self.bert = BertModel.from_pretrained(model_name) 
        # 線形変換を初期化しておく
        self.linear = torch.nn.Linear(
            self.bert.config.hidden_size, num_labels
        ) 

    def forward(
        self, 
        input_ids=None, 
        attention_mask=None, 
        token_type_ids=None, 
        labels=None
    ):
        # データを入力しBERTの最終層の出力を得る。
        bert_output = self.bert(
            input_ids=input_ids,
            attention_mask=attention_mask,
            token_type_ids=token_type_ids)
        last_hidden_state = bert_output.last_hidden_state
        
        # [PAD]以外のトークンで隠れ状態の平均をとる
        averaged_hidden_state = \
            (last_hidden_state*attention_mask.unsqueeze(-1)).sum(1) \
            / attention_mask.sum(1, keepdim=True)
        
        # 線形変換
        scores = self.linear(averaged_hidden_state) 
        
        # 出力の形式を整える。
        output = {'logits': scores}

        # labelsが入力に含まれていたら、損失を計算し出力する。
        if labels is not None: 
            loss = torch.nn.BCEWithLogitsLoss()(scores, labels.float())
            output['loss'] = loss
            
        # 属性でアクセスできるようにする。
        output = type('bert_output', (object,), output) 

        return output

In [5]:
# 7-13
class BertForSequenceClassificationMultiLabel_pl(pl.LightningModule):

    def __init__(self, model_name, num_labels, lr):
        super().__init__()
        self.save_hyperparameters() 
        self.bert_scml = BertForSequenceClassificationMultiLabel(
            model_name, num_labels=num_labels
        ) 

    def training_step(self, batch, batch_idx):
        output = self.bert_scml(**batch)
        loss = output.loss
        self.log('train_loss', loss)
        return loss
        
    def validation_step(self, batch, batch_idx):
        output = self.bert_scml(**batch)
        val_loss = output.loss
        self.log('val_loss', val_loss)

    def test_step(self, batch, batch_idx):
        labels = batch.pop('labels')
        output = self.bert_scml(**batch)
        scores = output.logits
        labels_predicted = ( scores > 0 ).int()
        num_correct = ( labels_predicted == labels ).all(-1).sum().item()
        accuracy = num_correct/scores.size(0)
        self.log('accuracy', accuracy)

    def configure_optimizers(self):
        return torch.optim.Adam(self.parameters(), lr=self.hparams.lr)


In [6]:
# 7-14
# 入力する文章
text_list = [
    "今期は売り上げが順調に推移したが、株価は低迷の一途を辿っている。",
    "次期の業績の見通しとしましては、売上高につきましては生産卸売事業、直販事業ともに増収を見込んでおります。",
    "農業にマイナス影響あるいは不透明感をもたらす状況が散見されております。"
]

# モデルのロード
#best_model_path = "model/izumi-epoch=4-step=699.ckpt"
best_model_path ="model/main.ckpt"
model = BertForSequenceClassificationMultiLabel_pl.load_from_checkpoint(best_model_path)
bert_scml = model.bert_scml#.cuda()

# データの符号化
encoding = tokenizer(
    text_list, 
    padding = 'longest',
    return_tensors='pt'
)
#encoding = { k: v.cuda() for k, v in encoding.items() }
encoding = { k: v for k, v in encoding.items() }

# BERTへデータを入力し分類スコアを得る。
with torch.no_grad():
    output = bert_scml(**encoding)
scores = output.logits
labels_predicted = ( scores > 0 ).int().cpu().numpy().tolist()

# 結果を表示
for text, label in zip(text_list, labels_predicted):
    print('--')
    print(f'入力：{text}')
    print(f'出力：{label}')

Some weights of the model checkpoint at cl-tohoku/bert-base-japanese-whole-word-masking were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight', 'cls.predictions.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


--
入力：今期は売り上げが順調に推移したが、株価は低迷の一途を辿っている。
出力：[1, 1]
--
入力：次期の業績の見通しとしましては、売上高につきましては生産卸売事業、直販事業ともに増収を見込んでおります。
出力：[0, 1]
--
入力：農業にマイナス影響あるいは不透明感をもたらす状況が散見されております。
出力：[0, 0]


## bertのモデル
-  'cl-tohoku/bert-base-japanese-whole-word-masking'

## ファインチューニング
- multi-hot-labelによる多クラス分類

## chabsa-data-setについて
- 文章と単語が与えられている．
- 単語を３つに分けている(pos,neutral,neg)
- 文章中でその単語がどのラベルに属しているかを表している．

注のある分は削除

# 出力
- [neg,pos]

In [7]:
# テキストの取得
#企業リストを取得
#企業リストを取得
company_list = pd.read_csv("../data/new2/df_achieve/company_list.csv",index_col=0)
company_list=company_list.drop(14350)
company_list=company_list.drop(52080)
company_list=company_list.drop(95080)
with open('../data/text_data_ver2/non_company_list.pkl', 'rb') as f:
    non_list = pickle.load(f)
for com in non_list:
    company_list=company_list.drop(com)

In [28]:
print(len(company_list))
company=company_list.index[0]
df_path = '../data/new2/df_achieve/this/{}.csv'.format(company)
df = pd.read_csv(df_path,index_col=0)

2355


In [14]:
code=df["pdf_code"][0]

In [15]:
json_path = '../data/text_data_ver2/text_data_sep_pre/'+str(company)+'/'+str(code)+'.json'
json_open = open(json_path, 'r')
json_load = json.load(json_open)

In [23]:
text_list=json_load["text"][2]["text"]

In [24]:
# データの符号化
encoding = tokenizer(
    text_list, 
    padding = 'longest',
    return_tensors='pt'
)
#encoding = { k: v.cuda() for k, v in encoding.items() }
encoding = { k: v for k, v in encoding.items() }

In [25]:
# BERTへデータを入力し分類スコアを得る。
with torch.no_grad():
    output = bert_scml(**encoding)
scores = output.logits
labels_predicted = ( scores > 0 ).int().cpu().numpy().tolist()

# 結果を表示
for text, label in zip(text_list, labels_predicted):
    print('--')
    print(f'入力：{text}')
    print(f'出力：{label}')

--
入力：当第0四半期連結累計期間におけるわが国経済は、雇用・所得環境の改善を背景に、個人消費が増加するなど、景気は緩やかな回復がみられるものの、中国を始めとする新興国経済の成長鈍化や保護主義の台頭を含む欧米の政治リスクなどが世界の実体経済に及ぼす影響が懸念され、依然として不透明な状況が続いております。
出力：[1, 1]
--
入力：水産・食品業界におきましては、食の安心・安全に対する消費者の関心は高く、さらに少子高齢化による国内マーケット環境の変化や人手不足による労働コストの上昇に加え、世界的な水産物需要の増大による買付コストの上昇など、厳しい状況は続いております。
出力：[1, 0]
--
入力：このような状況のもとで、中期経営計画「バリューアップ・キョクヨー0」の最終年度を迎え、「魚に強い総合食品会社として、収益基盤の安定と変化への対応力を高め、新たな価値を創造する企業を目指す」ことを基本方針とし、目標達成に向けて取り組んでおります。
出力：[0, 0]
--
入力：セグメント別の業績は次のとおりです。
出力：[0, 0]
--
入力：(水産商事セグメント)鮭鱒・エビ・北洋魚などの販売が順調に推移したほか、定塩鮭製品などの付加価値製品の拡販に努めました。
出力：[0, 1]
--
入力：また、海外での水産物販売についても米国マーケットでの販売が順調に伸びております。
出力：[0, 1]
--
入力：この結果、この部門は売上・利益ともに前年同期を上回りました。
出力：[0, 1]
--
入力：(冷凍食品セグメント)水産冷凍食品事業では寿司種を中心とした生食用商品及び「だんどり上手」シリーズなどの加熱用商品の拡販に努めました。
出力：[0, 0]
--
入力：また調理冷凍食品事業では自社工場製品の水産フライ類やカニ風味かまぼこの販売が伸長しました。
出力：[0, 1]
--
入力：家庭用冷凍食品事業では塩釜工場製品の販売を伸ばしております。
出力：[0, 1]
--
入力：この結果、この部門は売上・利益ともに前年同期を上回りました。
出力：[0, 1]
--
入力：(常温食品セグメント)サバやサンマなどの水産缶詰の拡販に努めるとともに、価格改定や商品集約を図りました。
出力：[0, 0]
--
入力：また、原料価格高騰が続いている海産珍味類は、価格改定や規格変

In [42]:
#最初からやり直し
non_dic={}
for company in tqdm.tqdm(company_list.index):
    df_path = '../data/new2/df_achieve/this/{}.csv'.format(company)
    #dfを呼び出し
    df = pd.read_csv(df_path,index_col=0)
    dic = '../data/text_data_ver2/text_and_bert_label_ver1/'+str(company)
    #フォルダを作成する
    try:
        os.makedirs(dic)
    except FileExistsError:
        pass
    
    non_dic[company]=[]
    #企業の決算短信を一つ一つ見る
    for code in df['pdf_code']:
        try:
            json_path = '../data/text_data_ver2/text_and_bert_label_ver0/'+str(company)+'/'+str(code)+'.json'
            out_path = '../data/text_data_ver2/text_and_bert_label_ver1/'+str(company)+'/'+str(code)+'.json'
            with open(json_path) as file:
                json_load= json.load(file)
                main_dic=json_load
                
            for i,text_dic in enumerate(json_load["all_text"]):
                if text_dic["text_label"]!=[]:
                    text_list=[text_dic["text_label"][k]["text"] for k in range(0,len(text_dic["text_label"]))]
                    # データの符号化
                    encoding = tokenizer(
                        text_list, 
                        padding = 'longest',
                        return_tensors='pt'
                    )
                    #encoding = { k: v.cuda() for k, v in encoding.items() }
                    encoding = { k: v for k, v in encoding.items() }

                    # BERTへデータを入力し分類スコアを得る。
                    with torch.no_grad():
                        output = bert_scml(**encoding)
                    scores = output.logits
                    labels_predicted = ( scores > 0 ).int().cpu().numpy().tolist()
                    
                    for j,label in enumerate(labels_predicted):
                        main_dic["all_text"][i]["text_label"][j]["label"]=label
                    
                    #保存
                    json_file = open(out_path, mode="w")
                    json.dump(main_dic,json_file, ensure_ascii=False)
                    json_file.close()
                    
        except:
            non_dic[company].append(code)

100%|████████████████████████████████████| 2355/2355 [31:07:24<00:00, 47.58s/it]


In [41]:
with open(out_path) as file:
    json_load= json.load(file)
json_load

{'all_text': [{'contents': '目次より前', 'text_label': []},
  {'contents': '１．当四半期決算に関する定性的情報', 'text_label': []},
  {'contents': '（１）経営成績に関する説明',
   'text_label': [{'text': '当第0四半期連結累計期間におけるわが国経済は、雇用・所得環境の改善を背景に、個人消費が増加するなど、景気は緩やかな回復がみられるものの、中国を始めとする新興国経済の成長鈍化や保護主義の台頭を含む欧米の政治リスクなどが世界の実体経済に及ぼす影響が懸念され、依然として不透明な状況が続いております。',
     'label': [1, 1]},
    {'text': '水産・食品業界におきましては、食の安心・安全に対する消費者の関心は高く、さらに少子高齢化による国内マーケット環境の変化や人手不足による労働コストの上昇に加え、世界的な水産物需要の増大による買付コストの上昇など、厳しい状況は続いております。',
     'label': [1, 0]},
    {'text': 'このような状況のもとで、中期経営計画「バリューアップ・キョクヨー0」の最終年度を迎え、「魚に強い総合食品会社として、収益基盤の安定と変化への対応力を高め、新たな価値を創造する企業を目指す」ことを基本方針とし、目標達成に向けて取り組んでおります。',
     'label': [0, 0]},
    {'text': 'セグメント別の業績は次のとおりです。', 'label': [0, 0]},
    {'text': '(水産商事セグメント)鮭鱒・エビ・北洋魚などの販売が順調に推移したほか、定塩鮭製品などの付加価値製品の拡販に努めました。',
     'label': [0, 1]},
    {'text': 'また、海外での水産物販売についても米国マーケットでの販売が順調に伸びております。', 'label': [0, 1]},
    {'text': 'この結果、この部門は売上・利益ともに前年同期を上回りました。', 'label': [0, 1]},
    {'text': '(冷凍食品セグメント)水産冷凍食品事業では寿司種を中心とし

In [8]:
#一部だけやり直し
non_dic={}
for company in tqdm.tqdm(company_list.index):
    df_path = '../data/new2/df_achieve/this/{}.csv'.format(company)
    #dfを呼び出し
    df = pd.read_csv(df_path,index_col=0)
    dic = '../data/text_data_ver2/bert_label/'+str(company)
    non_dic[company]=[]
    #企業の決算短信を一つ一つ見る
    for code in df['pdf_code']:
        try:
            json_path = '../data/text_data_ver2/text_and_bert_label/'+str(company)+'/'+str(code)+'.json'
            with open(json_path) as file:
                json_load= json.load(file)
                main_dic=json_load
                
            for i,text_dic in enumerate(json_load["all_text"]):
                if text_dic["text_label"]!=[] and text_dic["text_label"][0]["label"]==[-1,-1]:
                    text_list=[text_dic["text_label"][k]["text"] for k in range(0,len(text_dic["text_label"]))]
                    # データの符号化
                    encoding = tokenizer(
                        text_list, 
                        padding = 'longest',
                        return_tensors='pt'
                    )
                    #encoding = { k: v.cuda() for k, v in encoding.items() }
                    encoding = { k: v for k, v in encoding.items() }

                    # BERTへデータを入力し分類スコアを得る。
                    with torch.no_grad():
                        output = bert_scml(**encoding)
                    scores = output.logits
                    labels_predicted = ( scores > 0 ).int().cpu().numpy().tolist()
                    
                    for j,label in enumerate(labels_predicted):
                        main_dic["all_text"][i]["text_label"][j]["label"]=label
                    
                    #保存
                    json_file = open(json_path, mode="w")
                    json.dump(main_dic,json_file, ensure_ascii=False)
                    json_file.close()
                    
        except:
            non_dic[company].append(code)

100%|█████████████████████████████████████| 2355/2355 [1:41:08<00:00,  2.58s/it]


In [43]:
import requests

def main():
    url = "https://notify-api.line.me/api/notify"
    token = 'TySDNym3wzG4ab5jev51wZhOyCA20dFtxT0tkcIkpaF'
    headers = {"Authorization" : "Bearer "+ token}

    message =  'プログラムが終了しました．'
    payload = {"message" :  message}

    r = requests.post(url ,headers = headers ,params=payload)

if __name__ == '__main__':
    main()

In [45]:
for com in non_dic.keys():
    if non_dic[com]!=[]:
        print(com)

In [73]:
non_list=[]
for company in tqdm.tqdm(company_list.index):
    df_path = '../data/new2/df_achieve/this/{}.csv'.format(company)
    #dfを呼び出し
    df = pd.read_csv(df_path,index_col=0)
    dic = '../data/text_data_ver2/bert_label/'+str(company)
    #フォルダを作成する
    try:
        os.makedirs(dic)
    except FileExistsError:
        pass
        
    #企業の決算短信を一つ一つ見る
    for code in df['pdf_code']:
        try:
            json_path = '../data/text_data_ver2/text_data_sep_pre/'+str(company)+'/'+str(code)+'.json'
            out_pkl= '../data/text_data_ver2/bert_label/'+str(company)+'/'+str(code)+'.pkl'
            json_open = open(json_path, 'r')
            json_load = json.load(json_open)
            
            #テキストリストを少し処理
            text_list=json_load["text"][2]["text"]
            for i,text in enumerate(text_list):
                if len(text)<11:
                    text_list.pop(i)
                    continue

                elif text.count(',') >5:
                    text_list.pop(i)
                    continue

                elif "注" in text[:5]:
                    text_list.pop(i)
                    continue
                    
            # データの符号化
            encoding = tokenizer(
                text_list, 
                padding = 'longest',
                return_tensors='pt'
            )
            #encoding = { k: v.cuda() for k, v in encoding.items() }
            encoding = { k: v for k, v in encoding.items() }

            # BERTへデータを入力し分類スコアを得る。
            with torch.no_grad():
                output = bert_scml(**encoding)
            scores = output.logits
            labels_predicted = ( scores > 0 ).int().cpu().numpy().tolist()
            
            #保存
            with open(out_pkl, 'wb') as p:
                pickle.dump(labels_predicted, p)
        except:
            non_list.append(company)

100%|█████████████████████████████████████| 2510/2510 [9:43:41<00:00, 13.95s/it]


In [76]:
len(non_list)

8245

In [77]:
#保存
with open( '../data/text_data_ver2/bert_label/'+'non.pkl', 'wb') as p:
    pickle.dump(non_list, p)

In [70]:
with open(out_pkl, 'rb') as p:
        out_list=pickle.load(p)

In [71]:
out_list

[[1, 1],
 [1, 0],
 [0, 1],
 [0, 1],
 [0, 1],
 [0, 1],
 [0, 1],
 [0, 1],
 [0, 1],
 [0, 1],
 [0, 1],
 [0, 1],
 [1, 0],
 [0, 1],
 [1, 1],
 [0, 1],
 [1, 0],
 [1, 1],
 [0, 1],
 [0, 1]]

In [41]:
non_dic={}
# エラーを起こしたファイルを探す
for company in tqdm.tqdm(company_list.index):
    df_path = '../data/new2/df_achieve/this/{}.csv'.format(company)
    #dfを呼び出し
    df = pd.read_csv(df_path,index_col=0)
    dic = '../data/text_data_ver2/bert_label/'+str(company)
    file_list=os.listdir(dic)
    code_list = list(df["pdf_code"])
    non_dic[company]=[]
    for code in code_list:
        if str(code)+'.pkl' in file_list:
            continue
        
        else:
            non_dic[company].append(code)
    

100%|█████████████████████████████████████| 2510/2510 [00:01<00:00, 1323.23it/s]


In [42]:
non_dic

{13010: [],
 13320: [20210802476701, 20211104425228, 20220131576520],
 13330: [],
 13790: [20200515416026],
 13800: [],
 13810: [],
 13820: [],
 13830: [20171211433536],
 14000: [],
 14070: [],
 14140: [],
 14170: [],
 14180: [],
 14190: [],
 14200: [],
 14310: [],
 14330: [20200907489799, 20210312477677, 20220311504009],
 14340: [],
 14360: [20210614448164],
 14380: [],
 14390: [],
 14430: [],
 14460: [],
 14470: [],
 14510: [],
 14910: [],
 15140: [],
 15150: [20190523434658, 20210510408750, 20220510536417],
 15180: [],
 16050: [],
 16620: [20170804448287, 20171107410918, 20180201461875, 20180509430631],
 16630: [20200207460655],
 17110: [],
 17120: [20220404516755],
 17160: [],
 17180: [20180127457380],
 17190: [20170718433408,
  20171026499138,
  20180111449220,
  20180503427329,
  20180731488970,
  20190122462132,
  20190502414372,
  20200513411889,
  20210511412092,
  20220512545162],
 17200: [20180510432302, 20190423408693, 20200508407179, 20210511411540],
 17210: [],
 17240: [2

In [20]:
json_path = '../data/text_data_ver2/text_data_sep_pre/'+str(company)+'/'+str(code)+'.json'
json_open = open(json_path, 'r')
json_load = json.load(json_open)

True

In [40]:
#textのないデータには，空リストを保存する
cnt=0
for i,company in enumerate(company_list.index):
    if non_dic[company]!=[]:
        for code in non_dic[company]:
            json_path = '../data/text_data_ver2/text_data_sep_pre/'+str(company)+'/'+str(code)+'.json'
            json_open = open(json_path, 'r')
            json_load = json.load(json_open)
            if json_load["text"]==[]:
                out_pkl= '../data/text_data_ver2/bert_label/'+str(company)+'/'+str(code)+'.pkl'
                non_text=[]
                #保存
                with open(out_pkl, 'wb') as p:
                    pickle.dump(non_text, p)
                continue
            else:
                cnt+=1


In [44]:
for i,company in enumerate(company_list.index):
    if non_dic[company]==[]:
        
    
    else:
        
        for code in non_dic[company]:
            json_path = '../data/text_data_ver2/text_data_sep_pre/'+str(company)+'/'+str(code)+'.json'
            json_open = open(json_path, 'r')
            json_load = json.load(json_open)
            break
            #テキストリストを少し処理
            text_list=json_load["text"][2]["text"]
            for i,text in enumerate(text_list):
                if len(text)<11:
                    text_list.pop(i)
                    continue

                elif text.count(',') >5:
                    text_list.pop(i)
                    continue

                elif "注" in text[:5]:
                    text_list.pop(i)
                    continue
                    
            # データの符号化
            encoding = tokenizer(
                text_list, 
                padding = 'longest',
                return_tensors='pt'
            )
            #encoding = { k: v.cuda() for k, v in encoding.items() }
            encoding = { k: v for k, v in encoding.items() }

            # BERTへデータを入力し分類スコアを得る。
            with torch.no_grad():
                output = bert_scml(**encoding)
            scores = output.logits
            labels_predicted = ( scores > 0 ).int().cpu().numpy().tolist()
    

In [63]:
for text_list_kari in json_load["text"]:
    text_list=text_list_kari
    pop_list=[]
    for text in enumerate(text_list_kari["text"]):
        for i,text in enumerate(text_list):
                if 512<len(text)<11:
                    pop_list.append(i)
                    continue

                elif text.count(',') >5:
                    pop_list.append(i)
                    continue

                elif "注" in text[:5]:
                    pop_list.append(i)
                    continue
        #いらないテキストを削除         
        for pop in pop_list[::-1]:
            text_list.pop(pop)

In [66]:
for pop in pop_list[::-1]:
        print(pop)
        text_list.pop(pop)

5
4
3
2
1


In [69]:
text_list=["0","1","2","3","4","5","6","1","1"]

In [70]:
for pop in pop_list[::-1]:
            text_list.pop(pop)

In [71]:
text_list

['0', '6', '1', '1']

In [79]:
# データの符号化
encoding = tokenizer(
    text_list, 
    padding = 'longest',
    return_tensors='pt'
)
#encoding = { k: v.cuda() for k, v in encoding.items() }
encoding = { k: v for k, v in encoding.items() }

In [72]:
import requests

def main():
    url = "https://notify-api.line.me/api/notify"
    token = 'TySDNym3wzG4ab5jev51wZhOyCA20dFtxT0tkcIkpaF'
    headers = {"Authorization" : "Bearer "+ token}

    message =  'プログラムが終了しました．'
    payload = {"message" :  message}

    r = requests.post(url ,headers = headers ,params=payload)

if __name__ == '__main__':
    main()